In [9]:
import os
import pandas as pd
import numpy as np

class Participant():
    def __init__(self, par_num):
        self.par_num = par_num
        self.par_ID = f"participant_{self.par_num}"
        self.par_dir = os.path.join(os.getcwd(), "participants", self.par_ID)

        self.exp_order = self.get_exp_order()
        self.all_marker_timestamps = self.get_all_marker_timestamps(exp_order=self.exp_order)

    def get_exp_order(self):
        exp_order_filename = f"{self.par_ID}_experiment_order.txt"
        exp_order_filepath = os.path.join(self.par_dir, exp_order_filename)

        with open(exp_order_filepath) as f:
            lines = f.readlines()

        exp_order = []
        for line in lines:
            if "Block" in line or "-" in line or line == "\n":
                pass
            else:
                exp_order.append(line.strip("\n"))
        
        return exp_order

    def _parse_udp(self, udp):
        marker_ID_info = udp[2].strip(",").split("=")
        marker_ID_str = marker_ID_info[0]
        marker_ID = marker_ID_info[1]

        marker_val_info = udp[3].strip(",").split("=")
        marker_val_str = marker_val_info[0]
        marker_val = marker_val_info[1]

        marker_string_info = udp[4].strip(",").split("=")
        marker_string_str = marker_string_info[0]
        marker_string = marker_string_info[1]

        marker_ts_info = udp[5].strip("\n").split("=")
        marker_ts_str = marker_ts_info[0]
        marker_ts = marker_ts_info[1]

        marker_data = {marker_ID_str: marker_ID, marker_val_str: marker_val, marker_string_str: marker_string, marker_ts_str: marker_ts}
        
        return marker_data

    def parse_log_file(self, exp_name):
        log_dir = os.path.join(self.par_dir, exp_name, "data")
        for filename in os.listdir(log_dir):
            if ".log" in filename:
                log_filename = filename
        log_filepath = os.path.join(log_dir, log_filename)

        with open(log_filepath) as f:
            lines = f.readlines()

        udp_lines = []
        marker_data = {}
        for line in lines:
            if "UDP" in line:
                udp_lines.append(line.split("\t")[-1])

        marker_data = []
        try:
            start_udp = udp_lines[0].split(" ")
            marker_data.append(self._parse_udp(start_udp))
        except:
            print("ERROR", f"{exp_name}: Start marker not found!")
            marker_data.append("_")
        try:
            end_udp = udp_lines[1].split(" ")
            marker_data.append(self._parse_udp(end_udp))
        except:
            print("ERROR", f"{exp_name}: End marker not found!")
            marker_data.append("_")

        return marker_data

    def parse_task_order_file(self, exp_name):
        exp_dir = os.path.join(self.par_dir, exp_name)
        for filename in os.listdir(exp_dir):
            if ".csv" in filename:
                task_order_filename = filename
        task_order_filepath = os.path.join(exp_dir, task_order_filename)
        test = pd.read_csv(task_order_filepath)

        return test

    def get_data_filepath(self, exp_name):
        data_dir = os.path.join(self.par_dir, exp_name, "data")
        for filename in os.listdir(data_dir):
            if ".csv" in filename:
                data_filename = filename
        data_filepath = os.path.join(data_dir, data_filename)

        return data_filepath

    def csv_to_df(self, filepath):
        df = pd.read_csv(filepath)

        return df

    def get_all_marker_timestamps(self, exp_order):
        all_marker_timestamps = {}
        for exp_name in exp_order:
            start_marker, end_marker = self.parse_log_file(exp_name=exp_name)
            try:
                start_ts = start_marker["timestamp"]
            except:
                start_ts = "_"
            try:
                end_ts = end_marker["timestamp"]
            except:
                end_ts = "_"
            all_marker_timestamps[exp_name] = [start_ts, end_ts]

        return all_marker_timestamps

    def get_cols(self, df, cols):
        return df[cols]

    def parse_df(self, df, num_blocks, num_trials):
        df_by_block = {}
        for i in range(num_blocks):
            block_name = f"block_{i+1}"
            if i == 0:
                temp_df = df.iloc[i*num_trials:(i+1)*num_trials]
            else:
                temp_df = df.iloc[(i*num_trials)+i:((i+1)*num_trials)+i]  # skip Nan line between blocks
            df_by_block[block_name] = temp_df

        return df_by_block

class Audio_Narrative():
    def __init__(self, par):
        self.exp_name = "audio_narrative"
        self.data_filepath = par.get_data_filepath(exp_name=self.exp_name)
        self.marker_data = par.parse_log_file(exp_name=self.exp_name)
        self.df = par.csv_to_df(filepath=self.data_filepath)

        self.df = par.csv_to_df(filepath=self.data_filepath)
        cols = ["pieman_clip.started", "participant_response.text"]
        self.df_simp = par.get_cols(df=self.df, cols=cols)

class Go_No_Go():
    def __init__(self, par):
        self.exp_name = "go_no_go"
        self.num_blocks = 4
        self.num_trials = 20
        self.data_filepath = par.get_data_filepath(exp_name=self.exp_name)
        self.marker_data = par.parse_log_file(exp_name=self.exp_name)

        self.task_order = par.parse_task_order_file(exp_name=self.exp_name)
        self.task_order_simp = self._simp_task_order(task_order=self.task_order)

        self.df = par.csv_to_df(filepath=self.data_filepath)
        cols = ["match", "go_image.started", "go_resp.corr", "go_resp.rt"]
        self.df_simp = par.get_cols(df=self.df, cols=cols)
        self.df_by_block = par.parse_df(df=self.df_simp, num_blocks=self.num_blocks, num_trials=self.num_trials) 
    
    def _simp_task_order(self, task_order):
        task_order = task_order["task_order"].to_list()
        task_order_simp = [task.split("_")[0] for task in task_order]

        return task_order_simp

class King_Devick():  
    def __init__(self, par):
        self.exp_name = "king_devick"
        self.data_filepath = par.get_data_filepath(exp_name=self.exp_name)
        self.marker_data = par.parse_log_file(exp_name=self.exp_name)
        
        self.task_order = ["card_1", "card_2", "card_3"]
        
        self.df = par.csv_to_df(filepath=self.data_filepath)
        cols = ["card_image.started", "card_resp.rt"]
        self.df_simp = par.get_cols(self.df, cols)

class N_Back():
    def __init__(self, par):
        self.exp_name = "n_back"
        self.num_blocks = 9
        self.num_trials = 20
        self.data_filepath = par.get_data_filepath(exp_name=self.exp_name)
        self.marker_data = par.parse_log_file(exp_name=self.exp_name)
        
        self.task_order = par.parse_task_order_file(exp_name=self.exp_name)
        self.task_order_simp = self._simp_task_order(task_order=self.task_order)

        self.df = par.csv_to_df(filepath=self.data_filepath)
        cols = ["match", "stim_text.started", "stim_resp.corr", "stim_resp.rt"]
        self.df_simp = par.get_cols(df=self.df, cols=cols)
        self.df_by_block = par.parse_df(df=self.df_simp, num_blocks=self.num_blocks, num_trials=self.num_trials)

    def _simp_task_order(self, task_order):
        task_order = task_order["task_order"].to_list()
        task_order_simp = []
        for task in task_order:
            if "ZB" in task:
                temp = task.split("-")
                task_simp = f"{temp[0]}-{temp[1]}"
            else:
                task_simp = task.split("-")[0]
            task_order_simp.append(task_simp)
        
        return task_order_simp

class Resting_State():
    def __init__(self, par):
        self.exp_name = "resting_state"
        self.data_filepath = par.get_data_filepath(exp_name=self.exp_name)
        self.marker_data = par.parse_log_file(exp_name=self.exp_name)
        self.df = par.csv_to_df(filepath=self.data_filepath)

class Tower_of_London():
    def __init__(self, par):
        self.exp_name = "tower_of_london"
        self.num_blocks = 6
        self.num_trials = 6
        self.data_filepath = par.get_data_filepath(exp_name=self.exp_name)
        self.marker_data = par.parse_log_file(exp_name=self.exp_name)
        
        self.task_order = par.parse_task_order_file(exp_name=self.exp_name)
        self.task_order_simp = self._simp_task_order(task_order=self.task_order)

        self.df = par.csv_to_df(filepath=self.data_filepath)
        cols = ["match", "stim_image.started", "stim_resp.corr", "stim_resp.rt"]
        self.df_simp = par.get_cols(df=self.df, cols=cols)
        self.df_by_block = par.parse_df(df=self.df_simp, num_blocks=self.num_blocks, num_trials=self.num_trials) 
    
    def _simp_task_order(self, task_order):
        task_order = task_order["task_order"].to_list()
        task_order_simp = [task.split("_")[0] for task in task_order]

        return task_order_simp

class Video_Narrative_CMIYC(): 
    def __init__(self, par):
        self.exp_name = "video_narrative_cmiyc"
        self.data_filepath = par.get_data_filepath(exp_name=self.exp_name)
        self.marker_data = par.parse_log_file(exp_name=self.exp_name)
        self.df = par.csv_to_df(filepath=self.data_filepath)

        self.df = par.csv_to_df(filepath=self.data_filepath)
        cols = ["catchme_participant_response.text"]
        self.df_simp = par.get_cols(df=self.df, cols=cols)

class Video_Narrative_Sherlock():
    def __init__(self, par):
        self.exp_name = "video_narrative_sherlock"
        self.data_filepath = par.get_data_filepath(exp_name=self.exp_name)
        self.marker_data = par.parse_log_file(exp_name=self.exp_name)

        self.df = par.csv_to_df(filepath=self.data_filepath)
        cols = ["sherlock_participant_response.text"]
        self.df_simp = par.get_cols(df=self.df, cols=cols)

class vSAT():
    def __init__(self, par):
        self.exp_name = "vSAT"
        self.num_blocks = 4
        self.num_trials = 30
        self.data_filepath = par.get_data_filepath(exp_name=self.exp_name)
        self.marker_data = par.parse_log_file(exp_name=self.exp_name)

        self.task_order = par.parse_task_order_file(exp_name=self.exp_name)
        self.task_order_simp = self._simp_task_order(task_order=self.task_order)

        self.df = par.csv_to_df(filepath=self.data_filepath)
        cols = ["match", "stim_time", "x_pos", "y_pos", "vSAT_square.started", "stim_resp.corr", "stim_resp.rt"]
        self.df_simp = par.get_cols(df=self.df, cols=cols)
        self._add_pos_col()
        self.df_by_block = par.parse_df(df=self.df_simp, num_blocks=self.num_blocks, num_trials=self.num_trials) 

    def _simp_task_order(self, task_order):
        task_order = task_order["task_order"].to_list()
        task_order_simp = [task.split("_")[0] for task in task_order]

        return task_order_simp

    def _add_pos_col(self):
        x_pos_col = self.df_simp["x_pos"]
        y_pos_col = self.df_simp["y_pos"]

        pos_list = []
        for x_pos, y_pos in zip(x_pos_col, y_pos_col):
            if x_pos == 0 and y_pos == 0:
                pos = "center"
                pos_list.append(pos)
            elif x_pos == 0.25 and y_pos == 0.25:
                pos = "top-right"
                pos_list.append(pos)
            elif x_pos == 0.25 and y_pos == -0.25:
                pos = "bottom-right"
                pos_list.append(pos)
            elif x_pos == -0.25 and y_pos == 0.25:
                pos = "top-left"
                pos_list.append(pos)
            elif x_pos == -0.25 and y_pos == -0.25:
                pos = "bottom-left"
                pos_list.append(pos)
            else:
                pos = "Nan"
                pos_list.append(pos)
        self.df_simp.insert(loc=4, column="position", value=pos_list)
        self.df_simp.drop(columns=["x_pos", "y_pos"], inplace=True)

par_num = "03"
par = Participant(par_num=par_num)

audio_narrative = Audio_Narrative(par=par)
go_no_go = Go_No_Go(par=par)
king_devick = King_Devick(par=par)
n_back = N_Back(par=par)
resting_state = Resting_State(par=par)
tower_of_london = Tower_of_London(par=par)
video_narrative_cmiyc = Video_Narrative_CMIYC(par=par)
video_narrative_sherlock = Video_Narrative_Sherlock(par=par)
vsat = vSAT(par=par)

ERROR go_no_go: End marker not found!
ERROR go_no_go: End marker not found!


c:\Users\zackg\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
